In [5]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers=1, dropout=0.1):
        super(LSTMClassifier, self).__init__()
        
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout if num_layers > 1 else 0)
        self.hidden_size = hidden_dim
        self.num_layers = num_layers
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

        # Improved classifier head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, output_dim)
        )

        self.output = nn.Softmax(dim=-1)

    def forward(self, x):
        #h0 = torch.zeros(x.size(0), self.num_layers, self.hidden_size).to(x.device)
        #c0 = torch.zeros(x.size(0), self.num_layers,  self.hidden_size).to(x.device)
        #print("h0:", h0.shape)
        lstm_out, (h_n, c_n) = self.lstm(x)
        #print("lstm_out:", lstm_out.shape)
        final_hidden_state = lstm_out.reshape(lstm_out.size(0), lstm_out.size(1))
        print("final_hidden_state:", final_hidden_state.shape)
        print("layer_norm:", self.hidden_size)
        #print("final_hidden_state:", final_hidden_state.shape)
        pooled = self.layer_norm(final_hidden_state)
        pooled = self.dropout(pooled)

        # Classifier head
        out = self.classifier(pooled)
        #print("out:", pooled.shape)
        return self.output(out)

In [21]:
import torch
vocab_size = 10000
embedding_dim = 384
hidden_dim = 64
output_dim = 3  # For binary classification
sequence_length = 100

model = LSTMClassifier(vocab_size=vocab_size, embedding_dim=embedding_dim,hidden_dim = 32, output_dim = output_dim, num_layers=1, dropout=0.3)
model.load_state_dict(torch.load("lstm.pth", map_location='cuda'))
model.eval()

model = model.to('cuda')
# Load the embedding model
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2").to('cuda')

# Label mapping (example)
label_map = {0: "negative", 1: "neutral", 2: "positive"}

def predict_sentiment(text):
    inputs = embedding_model.encode(text, convert_to_tensor=True).unsqueeze(0).to('cuda')
    with torch.no_grad():
        print("inputs:", inputs.shape)
        outputs = model(inputs)
        prediction = torch.argmax(outputs, dim=1).item()
    return label_map[prediction]


/tmp/ipykernel_15495/892962628.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lstm.pth", map_location='cuda'))


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter text here..."),
    outputs=gr.Label(num_top_classes=3),
    title="Sentiment Classifier",
    description="Enter a sentence to predict its sentiment."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


inputs: torch.Size([1, 384])
final_hidden_state: torch.Size([1, 32])
layer_norm: 32
